<a href="https://colab.research.google.com/github/sccn/sound2meg/blob/main/Spatial_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import os
from torch.autograd import Variable
import math
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.optim as optim
from scipy.io import loadmat
from dataset_loading import Sound2MEGDataset
from torch.utils.data import Dataset, DataLoader, random_split

In [ ]:
#import scipy.io
#data = scipy.io.loadmat('/content/drive/MyDrive/data.mat')

In [ ]:
#x = data['data']
#x = x[:, 1:3600:10, :]
#print(x.shape)   

(273, 360, 7)


In [45]:
def cos_vector(k, K, x, y):
  cos_v = torch.zeros(273, K)
  for l in range(K):
    cos_v[:,l] = torch.cos(2*math.pi*(k*x+l*y))
  return cos_v
def sin_vector(k, K, x, y):
  sin_v = torch.zeros(273, K)
  for l in range(K):
    sin_v[:,l] = torch.sin(2*math.pi*(k*x+l*y))
  return sin_v

In [46]:
def SpatialAttentionFunc(in_channels, out_channels, X, z, K, cos, sin):
  a = torch.randn(out_channels, in_channels)
  #positions = loadmat('/content/drive/MyDrive/electrode_positions.mat')
  #positions = positions['positions']
  #x = torch.tensor(positions[:, 0])
  #y = torch.tensor(positions[:, 1])
  for j in range(out_channels):
    cos_sum = torch.zeros(in_channels)
    sin_sum = torch.zeros(in_channels)
    for k in range(K):
      z_r = z[j, k, :].real
      z_r = z_r.unsqueeze(0)
      z_i = z[j, k, :].imag
      z_i = z_i.unsqueeze(0)      
      #cos_k = torch.transpose(cos_vector(k, 32, x, y), 0, 1)
      #sin_k = torch.transpose(sin_vector(k, 32, x, y), 0, 1)
      cos_sum = cos_sum + torch.mm(z_r, cos[k])
      sin_sum = sin_sum + torch.mm(z_i, sin[k])
    a[j, :] = cos_sum + sin_sum 
  SA = torch.randn(out_channels, 360)
  for j in range(out_channels):
    exp1 = torch.mm(torch.exp(a[j, :]).unsqueeze(0), X)
    exp2 = torch.sum(torch.exp(a[j, 0:out_channels]))
    SA[j] = exp1/exp2
  return SA

In [47]:
class SubjectLayer(nn.Module):
  def __init__(self):
    super(SubjectLayer, self).__init__()
    self.layers = []

    for i in range(124): #124 subjects
      layer = nn.Conv2d(270, 270, 1)
      self.layers.append(layer)
      
  def forward(self, x, s_idx):
    for i in range(len(x)):
      x[i] = self.layers[s_idx[i]](x[i].clone())
    return x

In [ ]:
subject = SubjectLayer()

x = torch.randn(3, 270, 360, 1)

print(x.shape)
output = subject(x, [0, 1, 2, 3])
print(output.shape)

torch.Size([3, 270, 360, 1])
torch.Size([3, 270, 360, 1])


In [48]:
class SpatialAttention(nn.Module):
  def __init__(self,in_channels, out_channels, K, path):
    super(SpatialAttention, self).__init__()
    self.positions = loadmat(path + 'electrode_positions.mat')
    self.positions = self.positions['positions']
    self.x = torch.tensor(self.positions[:, 0])
    self.y = torch.tensor(self.positions[:, 1])
    self.cos = []
    self.sin = []
    for k in range(32):
      self.cos.append(torch.transpose(cos_vector(k, 32, self.x, self.y), 0, 1))
      self.sin.append(torch.transpose(sin_vector(k, 32, self.x, self.y), 0, 1))
    self.out = out_channels
    self.input = in_channels
    self.K = K
    self.z = Parameter(torch.randn(out_channels, K, K, dtype = torch.cfloat)/(32*32))
    self.z.requiresGrad = True
  def forward(self, X):
    N = X.size()[0]
    SA = torch.zeros(N, 270, 360)
    for i in range(N):
      SA[i] = SpatialAttentionFunc(self.input, self.out, X[i], self.z, self.K, self.cos, self.sin)
    return SA

In [49]:
class Net(nn.Module):
  def __init__(self, path):
    super(Net, self).__init__()
    self.SA = SpatialAttention(273, 270, 32, path)
    self.Subject = SubjectLayer()
  def forward(self, y, s_idx):
    x1 = self.SA(y).unsqueeze(0)
    x2 = torch.permute(x1, (1, 2, 3, 0)) # subject attention?
    x3 = nn.Conv2d(270, 270, (1, 1))(x2)
    x = self.Subject(x3, s_idx)
    for k in range(1,6):
      p = pow(2,(2*k)%5)
      q = pow(2,(2*k+1)%5)
      if k == 1:
        x = nn.Conv2d(270, 320, (3, 1), dilation = 1, padding = (1, 0))(x)
        x = nn.BatchNorm2d(320)(x)
        x = nn.GELU()(x)
        x = nn.Conv2d(320, 320, (3, 1), dilation = 1, padding = (1, 0))(x)
        x = nn.BatchNorm2d(320)(x)
        x = nn.GELU()(x)
        x = nn.Conv2d(320, 640, (3, 1), dilation = 2, padding = (2, 0))(x)
        x = torch.transpose(x, 3, 1)
        x = nn.GLU()(x)
        x = torch.transpose(x, 3, 1)
      else:
        x1 = nn.Conv2d(320, 320, (3, 1), dilation = p, padding = (p, 0))(x)
        x1 = nn.BatchNorm2d(320)(x1)
        x1 = nn.GELU()(x1)
        x2 = x + x1
        x3 = nn.Conv2d(320, 320, (3, 1), dilation = q, padding = (q, 0))(x2)
        x3 = nn.BatchNorm2d(320)(x3)
        x3 = nn.GELU()(x3)
        x4 = x2 + x2
        x_out = nn.Conv2d(320, 640, (3, 1), dilation = 2, padding = (2, 0))(x4)
        x_out = torch.transpose(x_out, 3, 1)
        x_out = nn.GLU()(x_out)
        x_out = torch.transpose(x_out, 3, 1)
    x_out = nn.Conv2d(320, 640, (1, 1))(x_out)
    x_out = nn.GELU()(x_out)
    x_out = nn.Conv2d(640, 120, (1, 1))(x_out)
    return x_out

In [39]:
def CLIP_loss(Z, Y):
  N = Y.size(dim = 0)
  inner_product = torch.zeros(N, N)
  log_softmax = torch.zeros(N)
  for i in range(N):
    for j in range(N):
      Z_row = torch.reshape(Z[i], (-1,))
      Y_row = torch.reshape(Y[j], (-1,))
      inner_product[i, j] = torch.dot(Z_row, Y_row)/(N*N)
  for i in range(N):
    inn = inner_product[i, :]
    log_softmax[i] = torch.log(nn.functional.softmax(inn, -1))[i]
  return sum(-1*log_softmax)

In [50]:
def CLIP_loss(Z, Y):
  N = Y.size(dim = 0)
  #inner_product = torch.zeros(N, N)
  log_softmax = torch.zeros(N)
  Z_row = torch.reshape(Z, (N, -1))
  Y_row = torch.reshape(Y, (N, -1))
  inner_product = torch.mm(Z_row, torch.transpose(Y_row, 1, 0))/(N*N)
  for i in range(N):
    inn = inner_product[i, :]
    log_softmax[i] = torch.log(nn.functional.softmax(inn, -1))[i]
  return sum(-1*log_softmax)

In [43]:
import sys
sys.tracebacklimit = 0

In [ ]:
Dataset = Sound2MEGDataset('/content/drive/MyDrive/sound2meg/')
training_data, validation_data, test_data = random_split(Dataset, [0.7, 0.2, 0.1], generator=torch.Generator().manual_seed(42))
Training_Data_Batches = DataLoader(training_data, batch_size = 2, shuffle = True)
BrainModule = Net('/content/drive/MyDrive/sound2meg/')
optimizer = optim.Adam(BrainModule.parameters(), lr = 0.0003)
loss_train = []
loss_val = []
for i in range(2):
  loss_t = 0
  for MEG, WAV, Sub in Training_Data_Batches:
    Sub = Sub.tolist()
    Z = BrainModule(MEG, Sub)
    Z = Z[:, :, :, 0]
    loss = CLIP_loss(Z, WAV.abs())
    torch.autograd.set_detect_anomaly(True)
    optimizer.zero_grad()
    loss.backward()
    print(loss.item())
    loss_t = loss_t + loss.item()
    optimizer.step()
  loss_train.append(loss_t/len(training_data))

1.3683106899261475
1.3808143138885498
1.380147933959961
1.3915014266967773
1.3883558511734009
1.3801331520080566


In [ ]:
print(len(validation_data))

3285


In [ ]:
loss_v = 0
for data in validation_data:
  MEG = data[0]
  WAV = data[1].abs()
  Sub = data[2].tolist()
  Z = BrainModule(MEG, Sub)
  Z = Z[:, :, :, 0]
  loss = CLIP_loss(Z, WAV)
  loss_v = loss_v + loss.item()

3
3
3
3
3


KeyboardInterrupt: ignored